In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time
import urllib
import sys
import re
from selenium import webdriver
import urllib.parse
#import peewee
#from peewee import *
#from playhouse.db_url import connect
from selenium.webdriver.chrome.options import Options
import os
import string as str
#import MySQLdb
#import mysql.connector
import pymysql
import socket
import traceback
import datetime
import datetime
from datetime import timedelta
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
cnx = pymysql.connect(user='jared', password='iscae100',host='140.96.83.145',port=3306,database='gsearch', charset='utf8') 

def save_data(data):
    global cnx
    cursor = cnx.cursor()
    #sql = "insert into Serena.google_result_temp(begindate,enddate,keyword,numresult) values(%(begindate)s,%(enddate)s,%(keyword)s,%(numresult)s)  ON DUPLICATE KEY UPDATE numresult=%(numresult)s  "
    sql = "insert into Serena.google_result_temp6week(begindate,enddate,keyword,numresult) values(%(begindate)s,%(enddate)s,%(keyword)s,%(numresult)s)  "

    try:
        print(data) 
        cursor.execute(sql,data)
        cnx.commit()
        
    except: 
        print('exception savedata')
        error_string=traceback.format_exc()
        #print(type(error_string))
        if "Duplicate entry" in error_string :  
            print('dupliacate pass')
            pass
        else:
            print(error_string)

def get_progress(query): #目前爬到哪一個日期
    global cnx
    cursor = cnx.cursor()
    sql = "select min(begindate) from Serena.google_result_temp6week where keyword=\""+query+"\"  "
    #resultdt=datetime.datetime.now().date()
    resultdt=datetime.datetime.strptime("2017-11-25","%Y-%m-%d").date()
    try:
#        print(sql)
        cursor.execute(sql)
        for u in cursor:
            resultdt=u[0]
    except: 
        print('exception get_progress')
        traceback.print_exc()
    return resultdt


#options = webdriver.ChromeOptions()
#options.add_argument("--headless")
#options.binary_location='/usr/bin/google-chrome'
#driver = webdriver.Chrome('/usr/bin/chromedriver',chrome_options=options)  # Optional argument, if not specified will search path.

#options.binary_location='C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe'
driver = webdriver.Chrome('C:\\Users\\ua50049\\Desktop\\chromedriver_win32\\chromedriver.exe')  # Optional argument, if not specified will search path.





def process_page(driver):
    html = driver.page_source
    soup = BeautifulSoup(html,"lxml")
    
    elmt=soup.find('div',{'id':'resultStats'})
    print(elmt.text)
    sresult=re.search(u'約有 (.+) 項結果',elmt.text)
    if sresult is None:
        sresult=re.search(u'(.+) 項結果',elmt.text)
    print(sresult)
    print(sresult.group(1))
    return sresult.group(1).replace(",","")


'''
def get_prevweek(dt):
    ed=dt.strftime("%m/%d/%Y").replace("/","%2F")
    dt=dt+timedelta(days=-6)
    bg=dt.strftime("%m/%d/%Y").replace("/","%2F")
    return (bg,ed)
'''
def get_prevweek(dt):
    ed=dt.strftime("%m/%d/%Y").replace("/","%2F")
    if dt == datetime.datetime.strptime("2016-11-12","%Y-%m-%d").date():
        dt=dt+timedelta(days=-34)
    else :
        dt=dt+timedelta(days=-41)
    bg=dt.strftime("%m/%d/%Y").replace("/","%2F")
    return (bg,ed)


def mysearch(dt,query):
    global enddate
    global driver
#    n= datetime.datetime.now()
    bg,ed=get_prevweek(dt)
    fullurl='https://www.google.com.tw/search?rlz=1C1CHZL_enTW725TW725&biw=1243&bih=666&tbs=cdr%3A1%2Ccd_min%3A'+bg+'%2Ccd_max%3A'+ed+'&ei=DLJKWqXSLoaA8QXv9LagCg&q='+query+'&oq='+query
    print(fullurl)
    driver.get(fullurl)
    try:
        #搜尋結果為0的時候ww這一行會timeout error,所以做except例外處理,給值:0
        ww = WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH, "//div[@id='resultStats'][string-length(text()) > 1]")))
        numresult=process_page(driver)
    except:
        numresult=0
    if dt == datetime.datetime.strptime("2016-11-12","%Y-%m-%d").date() :
        save_data({'begindate':dt+timedelta(days=-34),'enddate':dt,'keyword':query,'numresult':int(numresult)})
    else :
        save_data({'begindate':dt+timedelta(days=-41),'enddate':dt,'keyword':query,'numresult':int(numresult)})
    time.sleep(2)
    


enddate=datetime.datetime.strptime("2015-01-18","%Y-%m-%d")
n=datetime.datetime.strptime("2017-11-25","%Y-%m-%d")
#n=datetime.datetime.strptime("2016-09-15","%Y-%m-%d")



cursor = cnx.cursor()

print('select start:')   
   
sql="SELECT o.name, o.asCat FROM gsearch.search_list s, oneil.urcosme_prod_brand_google_segment o where s.id % 6 = 0 and o.brandid=s.pixid order by rand()"
cursor.execute(sql)
result=['深層卸妝油+DHC','mono eyes 眼影+Aritaum','單色眼影+ETUDE HOUSE','礦物控油蜜粉+innisfree','冰河醣蛋白保濕霜+契爾氏+or+KIEHLS','情挑誘光水唇膏+聖羅蘭+or+YSL'
,'蜂巢氣墊+珂莉奧+or+CLIO']


for u in cursor:   # 把要跑的keyword存到result list
    q=u[0]+'+'+ u[1]
    #result.append(q.replace("'",""))
    if ("KEVIN.MURPHY" not in q) and ("歐恩伊" not in q) and ("精油身體乳+專科" not in q) and("Sugar Scrub&Lip Balm" not in q): #精油身體乳+專科
        result.append(q)

print('result length:',len(result))
for query in result:  
    print(query)   
    n=get_progress(query)
    #n=None
    if n is None:
        n=datetime.datetime.strptime("2017-11-25","%Y-%m-%d").date()
        
    print('process get:',n)
    
    while n >= enddate.date():
        mysearch(n,query)
        if n == datetime.datetime.strptime("2016-11-12","%Y-%m-%d").date() :
            n=n+timedelta(days=-35)
        else :
            n=n+timedelta(days=-42)
   

print('close drive')
driver.close()
exit(1)

select start:
result length: 1285
深層卸妝油+DHC
process get: 2017-11-25
https://www.google.com.tw/search?rlz=1C1CHZL_enTW725TW725&biw=1243&bih=666&tbs=cdr%3A1%2Ccd_min%3A10%2F15%2F2017%2Ccd_max%3A11%2F25%2F2017&ei=DLJKWqXSLoaA8QXv9LagCg&q=深層卸妝油+DHC&oq=深層卸妝油+DHC
約有 4,810 項結果 (搜尋時間：0.41 秒) 
<_sre.SRE_Match object; span=(0, 12), match='約有 4,810 項結果'>
4,810
{'numresult': 4810, 'enddate': datetime.date(2017, 11, 25), 'keyword': '深層卸妝油+DHC', 'begindate': datetime.date(2017, 10, 15)}
https://www.google.com.tw/search?rlz=1C1CHZL_enTW725TW725&biw=1243&bih=666&tbs=cdr%3A1%2Ccd_min%3A09%2F03%2F2017%2Ccd_max%3A10%2F14%2F2017&ei=DLJKWqXSLoaA8QXv9LagCg&q=深層卸妝油+DHC&oq=深層卸妝油+DHC
約有 3,580 項結果 (搜尋時間：0.34 秒) 
<_sre.SRE_Match object; span=(0, 12), match='約有 3,580 項結果'>
3,580
{'numresult': 3580, 'enddate': datetime.date(2017, 10, 14), 'keyword': '深層卸妝油+DHC', 'begindate': datetime.date(2017, 9, 3)}
https://www.google.com.tw/search?rlz=1C1CHZL_enTW725TW725&biw=1243&bih=666&tbs=cdr%3A1%2Ccd_min%3A07%2F23%2F2017%2

In [25]:





sql="select * from oneil.urcosme_prod_brand_google_segment"
cursor.execute(sql)
result=[]
for u in cursor:   # 把要跑的keyword存到result list
    result.append(u)

In [4]:
    global cnx
    cursor = cnx.cursor()
    sql = "select min(enddate) from Serena.google_result_temp where keyword='"+query+"'  "
    #resultdt=datetime.datetime.now().date()
    resultdt=datetime.datetime.strptime("2017-12-30","%Y-%m-%d").date()
    try:
#        print(sql)
        cursor.execute(sql)
        for u in cursor:
            resultdt=u[0]
    except: 
        print('exception get_progress')
        traceback.print_exc()
    return resultdt

None


Traceback (most recent call last):
  File "<ipython-input-4-1ab31614b201>", line 2, in <module>
    print(e)
NameError: name 'e' is not defined


In [26]:
result

[('71815', '??????', 89, "???+or+L'OCCITANE", 'CE'),
 ('74914', '??????', 1458, '??+or+Freshel', 'CE'),
 ('81201', '???+?????', 21, 'DHC', 'E'),
 ('74950', '???????', 330, 'UNT', 'E'),
 ('73547', '??????', 8, '????+or+The Body Shop', 'CE'),
 ('78793', '???????', 3840, 'excel', 'E'),
 ('68557', '?????+?????', 199, '???+or+Cellina', 'CE'),
 ('66959', '????????', 3576, 'BYREDO', 'E'),
 ('74841', '????????+????', 3237, '?????+or+Kiss Me', 'CE'),
 ('66856', '???????', 38, '??+or+LANCOME', 'CE'),
 ('70441', '?????????', 826, 'PRADA', 'E'),
 ('68439', '??????????', 8, '????+or+The Body Shop', 'CE'),
 ('79604', '??????', 539, 'Solone', 'E'),
 ('64110', '????????', 909, 'BURBERRY', 'E'),
 ('76677', '?????+??', 33, '???+or+IPSA', 'CE'),
 ('73167', '?????????', 25, '???+or+ettusais', 'CE'),
 ('74722', '???????', 3747, 'WATERS', 'E'),
 ('74901', '?????????', 104, '?????+or+KOSE', 'CE'),
 ('67008', '????????', 119, 'LUSH', 'E'),
 ('77899', '??????????', 3752, '???+or+Erborian', 'CE'),
 ('77694', '?

In [16]:
from selenium import webdriver
import time

chrome_path = "C:\\Users\\ua50049\\Desktop\\chromedriver_win32\\chromedriver.exe" #chromedriver.exe執行檔所存在的路徑
web = webdriver.Chrome(chrome_path)



cursor = cnx.cursor()
sql="SELECT distinct keyword FROM gsearch.googleresult"
cursor.execute(sql)
result=[]
for u in cursor:   # 把要跑的keyword存到result list
    result.append(u[0])

print(result)
for query in result:  

    #query="亮頰透漾腮紅+1028"
    
    n=get_progress(query)
    if n is None:
        n=datetime.datetime.strptime("2017-12-30","%Y-%m-%d").date()
        
    print(n)
    
    while n >= enddate.date():
        mysearch(n,query)
        n=n+timedelta(days=-7)



web.get('https://www.google.com.tw/search?q=1028+%E7%84%A1%E6%A5%B5%E9%99%90%E9%BB%91%E7%B5%A8&oq=1028&aqs=chrome.2.69i59l3j69i60j69i59j69i60.2724j0j4&sourceid=chrome&ie=UTF-8')
web.set_window_position(0,0) #瀏覽器位置
web.set_window_size(700,700) #瀏覽器大小
time.sleep(5)

web.find_element_by_link_text('1028 無極限黑絨商品價格- FindPrice 價格網').click() #點擊頁面上"天氣預報"的連結
time.sleep(5)

cursor = cnx.cursor()
sql="SELECT distinct keyword FROM gsearch.googleresult"
cursor.execute(sql)
result=[]
for u in cursor:   # 把要跑的keyword存到result list
    result.append(u[0])

print(result)web.close()

OSError: [WinError 6] 控制代碼無效。

In [28]:

import numpy as np
### for MySQL DB
conn = pymysql.connect(host='140.96.83.145', user='jared', passwd='iscae100', db='gsearch', charset='utf8')
curGet = conn.cursor()
q = """select * from oneil.urcosme_prod_brand_google_segment """
curGet.execute(q)
dataGet=[]
for readline in curGet:
    list = []
    list.extend(readline)
    dataGet.append(list)

dataGetArray = np.array(dataGet)

In [29]:
dataGetArray

array([['71815', '草本淨化皂泥', '89', "歐舒丹+or+L'OCCITANE", 'CE'],
       ['74914', '美肌淨透蜜粉', '1458', '膚蕊+or+Freshel', 'CE'],
       ['81201', '眼影蜜+瑪麗貓限定', '21', 'DHC', 'E'],
       ..., 
       ['73195', '安全之吻唇膏', '1200', 'absolution', 'E'],
       ['71479', '週年紀念晶燦寶盒', '254', '吉麗絲朵+or+JILL STUART', 'CE'],
       ['72322', '智能活水精華', '365', '葆麗美+or+SUISSE PROGRAMME', 'CE']], 
      dtype='<U34')

In [22]:
# -*- coding: utf-8 -*-   
import  sys  
import  re  

s="愛海花妍海浴乳(檸檬/鼠尾草) CREAM WASH(LEMON&SAGE)"
#unicode chinese  
re_words = re.compile("[\u4e00-\u9fa5]+" )  

res = re.findall(re_words, s)        #查詢出所有的匹配字符串  

print(res)


['愛海花妍海浴乳', '檸檬', '鼠尾草']
